# 나만의 사고 AI 서비스 '🚀 사고닷 🚀'
사고 관련 법률 상담을 AI를 활용하여 자동화하고,    
단계별 질문을 통해 사용자 맞춤형 법률 정보를 제공   


In [1]:
!pip install --upgrade jsonlines openai langchain langchain-openai langchain-community beautifulsoup4 langchain_chroma chromadb==0.5.3

Defaulting to user installation because normal site-packages is not writeable


In [2]:
!pip install pymupdf sentence-transformers faiss-cpu


Defaulting to user installation because normal site-packages is not writeable


In [ ]:
import requests
from bs4 import BeautifulSoup
import os

#os.environ["OPENAI_API_KEY"] = "<OpenAI_API의 API 키>"
os.environ["OPENAI_API_KEY"] = "YOUR_KEY"

os.environ['USER_AGENT']='MyCustomAgent'
# 아래 코드 Warning 제거

In [4]:
import fitz  # PyMuPDF 라이브러리

def extract_text_from_pdf(pdf_path):
    text = ""
    doc = fitz.open(pdf_path)  # PDF 열기
    for page in doc:
        text += page.get_text("text") + "\n"
    return text

# 저장된 PDF 파일들 처리
pdf_folder = "pdf_files"
pdf_texts = {}

for pdf_file in os.listdir(pdf_folder):
    pdf_path = os.path.join(pdf_folder, pdf_file)
    text = extract_text_from_pdf(pdf_path)
    pdf_texts[pdf_file] = text
    print(f"텍스트 추출 완료: {pdf_file}")


텍스트 추출 완료: 한글 6법전(헌법, 민법, 형법, 상법, 민사소송법, 형사소송법).pdf
텍스트 추출 완료: 한글 4법전(헌법, 민법, 형법, 형사소송법).pdf


In [6]:
import faiss
import numpy as np
from sentence_transformers import SentenceTransformer

# 임베딩 모델 로드 (빠르고 가벼운 모델)
model = SentenceTransformer("all-MiniLM-L6-v2")

# 벡터 변환
pdf_embeddings = {}
for pdf_file, text in pdf_texts.items():
    embedding = model.encode(text)
    pdf_embeddings[pdf_file] = embedding
    print(f"벡터 변환 완료: {pdf_file}")

# FAISS 벡터 DB 생성
embedding_dim = len(next(iter(pdf_embeddings.values())))  # 벡터 차원
index = faiss.IndexFlatL2(embedding_dim)  

# 벡터 삽입
pdf_files = list(pdf_embeddings.keys())
pdf_vectors = np.array(list(pdf_embeddings.values()), dtype="float32")
index.add(pdf_vectors)

# FAISS 인덱스 저장
faiss.write_index(index, "faiss_index.bin")
np.save("pdf_files.npy", np.array(pdf_files))  # PDF 파일명 저장

print("FAISS DB 저장 완료 ✅")


벡터 변환 완료: 한글 6법전(헌법, 민법, 형법, 상법, 민사소송법, 형사소송법).pdf
벡터 변환 완료: 한글 4법전(헌법, 민법, 형법, 형사소송법).pdf
FAISS DB 저장 완료 ✅


In [ ]:
import os
from openai import OpenAI
from langchain_openai import ChatOpenAI
from langchain_community.retrievers import TavilySearchAPIRetriever
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain.schema import Document


# OpenAI 클라이언트 연결
client = OpenAI(api_key="YOUR_KEY")
os.environ["TAVILY_API_KEY"] = "JINSIL'S_KEY"


class Agent:
    def __init__(self, system_prompt=""):
        self.system_prompt = system_prompt
        self.messages = [{"role": "system", "content": system_prompt}]  # 초기 프롬프트 포함
        self.user_queries = []

    def __call__(self, message):
        """사용자의 입력을 받아 AI 응답을 생성"""
        self.messages.append({"role": "user", "content": message})
        self.user_queries.append(message)

        result = self.execute()
        self.messages.append({"role": "assistant", "content": result})
        return result

    def execute(self):
        """AI에게 메시지를 보내고 응답을 받음"""
        completion = client.chat.completions.create(
            model="gpt-4o-mini",
            temperature=0,
            messages=self.messages
        )
        return completion.choices[0].message.content

    def summarize_conversation(self):
        """사용자 상담 내용을 한 문장으로 요약"""
        summary_prompt = f"""
        다음 사용자의 질문을 한 문장으로 요약하세요: {self.user_queries}
        """
        print(self.user_queries)
        
        completion = client.chat.completions.create(
            model="gpt-4o-mini",
            temperature=0,
            messages=[{"role": "system", "content": "대화 내용을 한 문장으로 요약하는 역할을 수행합니다."},
                      {"role": "user", "content": summary_prompt}]
        )
        return completion.choices[0].message.content
    

# **시스템 프롬프트 (단계별 상담 + 가이드라인 유지)**
prompt = """
당신은 사고 관련 법률 정보를 제공하는 AI 법률 상담사입니다.
사용자가 사고와 관련된 법률적 도움을 요청할 때, 아래의 단계별 질문 프레임워크를 따라 포괄적인 질문에서 구체적인 질문으로 진행하며 정확한 법률 정보를 제공해야 합니다.

# 시스템 지침
1. 사용자의 첫 질문이나 문의를 들은 후, 단계별로 질문을 진행하세요.
2. 각 단계마다 답변을 듣고 그에 맞는 다음 단계의 질문을 선택하세요.
3. 사용자의 응답에 따라 관련 법률 정보와 조언을 제공하세요.
4. 모든 법률 정보는 정확하고 최신의 정보여야 합니다.
5. 전문적이지만 이해하기 쉬운 언어로 소통하세요.
6. 사용자가 추가 질문이나 명확한 설명을 요청할 경우, 즉시 응답하세요.
7. 법률적 조언이 필요한 경우 실제 변호사와 상담할 것을 권장하세요.

# 응답 가이드라인
1. 사용자의 응답이 불분명하거나 부족한 경우, 추가 질문을 통해 명확히 하세요.
2. 사용자가 질문 단계를 건너뛰거나 다른 주제로 전환하려 할 경우, 필요한 정보를 얻기 위해 질문을 다시 유도하세요.
3. 사용자가 특정 법률 용어나 절차에 대해 이해하지 못하는 경우, 이해하기 쉽게 설명해주세요.
4. 유사한 판례나 법률 조항을 인용할 때는 정확한 출처와 내용을 제공하세요.
5. 모든 정보와 조언은 객관적이고 사실에 기반해야 합니다.
6. 사용자의 감정적 상태를 고려하여 공감적인 태도를 유지하세요.
7. 사용자의 상황이 긴급하거나 심각한 경우, 적절한 긴급 조치나 전문가 상담을 권장하세요.

# 질문 프로세스
1. Thought: 질문을 분석하여 어떤 정보를 더 수집해야 하는지 결정하세요.
2. Action: 필요한 정보를 얻기 위한 질문을 생성하세요.
3. 사용자 응답 후 다음 단계로 진행하세요.
"""

# AI 상담 Agent 생성
abot = Agent(system_prompt=prompt)

# 🔹 Tavily 검색 관련 설정
def search_law_cases(query):
    """Tavily에서 실제 판례 기사 검색 및 요약"""
    tavily = TavilySearchResults()
    result = tavily.invoke(query)

    # 검색 결과를 하나의 문서로 정리
    full_content = ""
    for item in result:
        full_content += f"📌 **{item['title']}**\n"
        full_content += f"{item['content']}\n"
        full_content += f"[🔗 링크]({item['url']})\n\n"

    return full_content

def format_docs(docs):
    """Tavily 검색 결과를 정리하는 함수"""
    formatted_docs = ""
    for d in docs:
        # d.page_content는 텍스트 내용, d.metadata에 URL이 포함되어 있을 수 있습니다.
        formatted_docs += f"Content: {d.page_content}\n"  # 문서 내용
        if hasattr(d, 'metadata') and 'source' in d.metadata:
            formatted_docs += f"URL: {d.metadata['source']}\n"  # 문서 URL
        formatted_docs += "\n"  # 항목 간 구분
    return formatted_docs

# 🔹 Tavily 검색을 위한 LLM 프롬프트
template = """Answer the question based only on the following context:

{context}

Question: {question}

사용자에게 얻은 정보에 따라 웹을 검색하고, 검색된 사건 관련 법률 정보와 사례를 아래 형식에 맞게 정리하여 답변을 생성해주세요.
<법률 정보>
법률 제목: 절도에 관한 형법
법률 조항: "타인의 재산을 불법적으로 취득한 경우, 절도로 간주되어 형사 처벌을 받을 수 있습니다."
법률 요약: 절도는 타인의 재산을 불법적으로 취득하는 행위로, 이 법은 절도의 정의 및 처벌 내용을 명시하고 있습니다.
법률 제목: 형법 제329조 (절도)
법률 조항: "절도의 경우, 피해 금액에 따라 처벌이 달라지며, 피해자가 10만원 이상을 잃은 경우 형사 처벌을 받게 됩니다."
법률 요약: 이 법은 절도의 처벌 기준을 정하며, 금액에 따라 다르게 처벌됩니다. 특히, 절도 금액이 10만원 이상일 경우, 형사 처벌이 부과됩니다.
<관련 사례>
사례 제목: 서울 절도 사건 배상 책임 사례
사례 요약: "2020년에 발생한 서울의 한 절도 사건에서, 피고인은 50만원 규모의 절도로 유죄 판결을 받았으며, 금전적 배상 및 징역형을 선고받았습니다."
링크: 외부로부터 받아오는 문서의 'URL' 정보를 링크로 달아주세요.
사례 제목: 절도 사건의 판결: 피해자 배상 사례
사례 요약: "2019년 발생한 절도 사건에서, 피해자는 가해자에게 법적 배상을 청구했습니다. 법원은 피해 금액을 기준으로 배상액을 산정하며, 피해자가 입은 피해를 보상받게 했습니다."
링크: 외부로부터 받아오는 문서의 'URL' 정보를 링크로 달아주세요.
사례 제목: 절도 사건의 합의 판결 사례
사례 요약: "2021년에 발생한 절도 사건에서, 피해자는 가해자와 합의를 통해 금전적 배상금을 받기로 했으며, 법원은 그 합의를 인정하여 사건을 종결했습니다."
링크: 외부로부터 받아오는 문서의 'URL' 정보를 링크로 달아주세요.
<예상 결과>
승률: "절도 사건의 승률은 피해 금액과 증거에 따라 다르며, 현 상황에서는 50만원 규모의 절도로 유죄 판결을 받을 것으로 예상됩니다.."
형량: "형량은 범죄의 심각성과 피해 금액에 따라 다르며, 일반적으로 금전적 배상과 함께 형사 처벌이 부과됩니다."
소요 기간: "절도 사건의 경우 보통 3개월에서 6개월 정도 소요될 수 있으며, 피해자가 법적 배상을 청구한 경우 기간이 더 길어질 수 있습니다.">
"""
prompt_template = ChatPromptTemplate.from_template(template)
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
retriever = TavilySearchAPIRetriever(k=3, search_depth="advanced", include_domains=["news"])

chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt_template
    | llm
)


def interactive_law_consultation():
    """사용자가 질문하면 AI가 단계별 질문을 진행하며 정보를 수집"""
    print("\n[나만의 사고 AI 서비스 '🚀 사고닷 🚀' 법률 상담 챗봇] 상담을 시작합니다. (종료하려면 'exit' 입력)\n")

    # 초기 사용자 질문 입력
    while True:
        user_query = input("사용자: ")

        if user_query.lower() == "exit":
            print("\n[🚀 사고닷 🚀] 상담을 종료합니다.")
            
            # 🔹 사용자 상담 내용을 요약하여 Tavily 검색 수행
            summary = abot.summarize_conversation()
            print("\n🚀 상담 요약:", summary)

            # 🔹 Tavily 검색 실행
            law_search_result = search_law_cases(f"{summary} 관련 실제 판례 기사 내용")
            print("\n🔎 검색된 법률 정보:\n", law_search_result)

            # 🔹 검색된 내용을 Tavily AI로 요약
            ai_search_result = chain.invoke(f"{summary} 관련 형량이나 벌금 정보")
            print("\n📜 Tavily AI 검색 요약:\n", ai_search_result.content)

            break

        ai_response = abot(user_query)
        print("\n🚀 사고닷 🚀 : ", ai_response)


# **실행**
interactive_law_consultation()



[나만의 사고 AI 서비스 '🚀 사고닷 🚀' 법률 상담 챗봇] 상담을 시작합니다. (종료하려면 'exit' 입력)


🚀 사고닷 🚀 :  전세 사기를 당하셨다니 매우 안타깝습니다. 이 문제를 해결하기 위해 몇 가지 질문을 드리겠습니다.

첫 번째로, 전세 사기를 당한 상황에 대해 좀 더 구체적으로 말씀해 주실 수 있나요? 예를 들어, 어떤 방식으로 사기를 당하셨는지, 계약서가 있었는지, 그리고 현재 어떤 조치를 취하고 계신지 알려주시면 좋겠습니다.

🚀 사고닷 🚀 :  주인과 연락이 되지 않아 보증금을 돌려받지 못하는 상황이시군요. 이 경우, 몇 가지 추가 질문을 통해 상황을 더 명확히 파악해 보겠습니다.

1. 계약서에는 보증금 반환에 대한 조항이 어떻게 되어 있나요? 
2. 주인과의 연락이 끊어진 지 얼마나 되었나요? 
3. 주인의 연락처 외에 다른 방법으로 연락을 시도해 보셨나요? (예: 친구나 가족을 통해, 또는 주인의 주소로 편지를 보내는 등)
4. 전세 계약이 언제 종료되었는지, 그리고 보증금 반환 기한이 있었는지 알려주실 수 있나요? 

이 정보를 통해 더 구체적인 조언을 드릴 수 있을 것 같습니다.

🚀 사고닷 🚀 :  좋습니다. 전세 계약서의 보증금 반환 조항은 매우 중요합니다. 일반적으로 계약서에는 다음과 같은 내용이 포함되어야 합니다:

1. **보증금 반환 시기**: 계약 종료 후 보증금을 언제 반환해야 하는지 명시되어 있어야 합니다. 예를 들어, "계약 종료 후 30일 이내에 보증금을 반환한다"는 식으로요.

2. **보증금 반환 조건**: 보증금을 반환하기 위한 조건이 명시되어 있을 수 있습니다. 예를 들어, "세입자가 집을 원상복구한 경우" 또는 "세입자가 계약을 성실히 이행한 경우"와 같은 조건이 있을 수 있습니다.

3. **연락처 및 통지 방법**: 보증금을 반환하기 위해 주인에게 연락할 때 어떤 방법으로 통지해야 하는지에 대한 내용이 있을 수 있습니다. 예를 들어, "서면으로 통지해야 한다"는 조항이 있을 수 있습니다.

4